# 🧠 Physics-Informed Neural Operators (PINO) 概述教程

本教程将带你全面了解 **PINO (Physics-Informed Neural Operators)** 的核心概念与实现方法。

---

## 📋 目录

1. **什么是 PINO？** - 基本概念介绍
2. **PINO 的核心思想** - 算子学习 + 物理约束
3. **与 PINNs 和 FNO 的对比** - 三种方法的异同
4. **数学基础** - Fourier 神经算子架构
5. **损失函数设计** - 数据损失 + 物理损失
6. **代码实现** - PyTorch 完整实现
7. **训练与评估** - Burgers 方程实战
8. **总结与展望** - 未来发展方向

---

## 🎯 学习目标

完成本教程后，你将能够：

- ✅ 理解 PINO 的核心思想和优势
- ✅ 掌握 Fourier Neural Operator 的工作原理
- ✅ 实现 PINO 模型并求解简单 PDE
- ✅ 设计合理的物理信息损失函数

## 1. 什么是 PINO？

**PINO (Physics-Informed Neural Operators)** 是一种结合了**神经算子**和**物理信息约束**的深度学习方法，由 Li et al. (2021) 提出。

### 核心定义

PINO 学习一个**参数化 PDE 的解算子**：

$$
\mathcal{G}_\theta: \mathcal{A} \to \mathcal{U}
$$

其中：
- $\mathcal{A}$: 输入函数空间（如初始条件、边界条件、PDE 系数）
- $\mathcal{U}$: 输出函数空间（PDE 解）
- $\theta$: 神经网络参数

### 关键创新

PINO 的核心创新在于**同时利用数据和物理约束**来训练神经算子：

$$
\mathcal{L}_{PINO} = \underbrace{\mathcal{L}_{data}}_{\text{数据拟合}} + \lambda \underbrace{\mathcal{L}_{PDE}}_{\text{物理残差}}
$$

这种设计使得 PINO：
- 比 **FNO** 需要更少的标签数据
- 比 **PINNs** 具有更强的泛化能力
- 保持了物理一致性

## 2. PINO 与 PINNs、FNO 的对比

在深入学习 PINO 之前，让我们先了解它与两种相关方法的区别：

### 2.1 PINNs (Physics-Informed Neural Networks)

**PINNs** 训练神经网络直接拟合单个 PDE 的解 $u(x, t)$：

$$
\text{PINNs}: \mathcal{F}_\theta(x, t) \to u(x, t)
$$

**优点**：无需标签数据，只需物理方程  
**缺点**：每个新问题需要重新训练

### 2.2 FNO (Fourier Neural Operator)

**FNO** 学习从输入函数到解函数的映射算子：

$$
\text{FNO}: u_0(x) \to u(x, T)
$$

**优点**：训练后可快速推理新问题  
**缺点**：需要大量标签数据

### 2.3 PINO = 算子学习 + 物理约束

**PINO** 结合了两者的优点：

| 特性 | PINNs | FNO | PINO |
|------|-------|-----|------|
| 学习目标 | 单一解 | 解算子 | 解算子 |
| 物理约束 | ✅ 强 | ❌ 无 | ✅ 有 |
| 泛化能力 | ❌ 弱 | ✅ 强 | ✅ 强 |
| 数据需求 | 低 | 高 | 中等 |
| 推理速度 | 慢 | 快 | 快 |

## 3. 导入所需库

首先，让我们导入实现 PINO 所需的所有库。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子以保证可重复性
torch.manual_seed(42)
np.random.seed(42)

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  使用设备: {device}")
print(f"📦 PyTorch 版本: {torch.__version__}")

# 绘图风格设置
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

## 4. Fourier Neural Operator (FNO) 层

PINO 的核心组件是 **Fourier Neural Operator (FNO)** 层。FNO 的关键思想是在**频域**进行学习，利用 FFT 的高效性。

### 4.1 谱卷积层 (Spectral Convolution)

FNO 的核心是谱卷积层，它在 Fourier 空间中执行线性变换：

$$
(\mathcal{K}v)(x) = \mathcal{F}^{-1}(R \cdot \mathcal{F}(v))(x)
$$

其中：
- $\mathcal{F}$: 傅里叶变换
- $\mathcal{F}^{-1}$: 逆傅里叶变换
- $R$: 可学习的频域权重矩阵（只保留低频模式）

### 为什么使用 Fourier 变换？

1. **全局感受野**: 可以捕获长距离依赖
2. **高效计算**: FFT 的复杂度为 $O(N \log N)$
3. **适合平滑函数**: PDE 解通常是平滑的

In [ ]:
class SpectralConv1d(nn.Module):
    """
    1D 谱卷积层 (Fourier Layer)
    
    核心操作:
    1. 输入 → FFT → 频域表示
    2. 频域表示 × 可学习权重 (只保留低频)
    3. 结果 → IFFT → 输出
    
    参数:
        in_channels: 输入通道数
        out_channels: 输出通道数
        modes: 保留的 Fourier 模式数 (低频分量)
    """
    
    def __init__(self, in_channels, out_channels, modes):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes = modes  # 保留的 Fourier 模式数
        
        # 初始化可学习的 Fourier 系数 (复数)
        self.scale = 1 / (in_channels * out_channels)
        self.weights = nn.Parameter(
            self.scale * torch.rand(in_channels, out_channels, modes, dtype=torch.cfloat)
        )
    
    def forward(self, x):
        """
        输入: x [batch, channels, spatial]
        输出: [batch, out_channels, spatial]
        """
        batch_size = x.shape[0]
        
        # Step 1: 计算 FFT
        x_ft = torch.fft.rfft(x)
        
        # Step 2: 在低频模式上乘以可学习权重
        out_ft = torch.zeros(batch_size, self.out_channels, x.size(-1) // 2 + 1,
                            dtype=torch.cfloat, device=x.device)
        out_ft[:, :, :self.modes] = torch.einsum(
            "bix,iox->box", x_ft[:, :, :self.modes], self.weights
        )
        
        # Step 3: 逆 FFT 回到空间域
        x = torch.fft.irfft(out_ft, n=x.size(-1))
        return x


class FNOBlock1d(nn.Module):
    """
    FNO 基本块 = 谱卷积 + 局部卷积 + 残差连接
    
    结构:
        input ─┬─ SpectralConv ─┬─ + ─ BatchNorm ─ GELU ─ output
               └─── Conv1x1 ────┘
    """
    
    def __init__(self, width, modes):
        super().__init__()
        self.spectral_conv = SpectralConv1d(width, width, modes)
        self.w = nn.Conv1d(width, width, 1)  # 局部线性变换
        self.bn = nn.BatchNorm1d(width)
    
    def forward(self, x):
        # 并行分支
        x1 = self.spectral_conv(x)  # 全局 (频域)
        x2 = self.w(x)              # 局部 (空间域)
        
        # 残差连接
        x = x1 + x2
        x = self.bn(x)
        x = F.gelu(x)
        return x


# 测试 FNO 层
print("✅ FNO 层测试:")
test_input = torch.randn(4, 32, 128)  # [batch, channels, spatial]
fno_block = FNOBlock1d(width=32, modes=16)
test_output = fno_block(test_input)
print(f"   输入形状: {test_input.shape}")
print(f"   输出形状: {test_output.shape}")

## 5. 构建 PINO 模型架构

完整的 PINO 模型包含三个部分：

1. **Lifting Layer**: 将输入提升到高维特征空间
2. **FNO Blocks**: 堆叠多个 Fourier 层进行特征变换
3. **Projection Layer**: 将高维特征投影回输出空间

$$
\text{输入} \xrightarrow{\text{Lifting}} \text{高维特征} \xrightarrow{\text{FNO} \times L} \text{变换特征} \xrightarrow{\text{Projection}} \text{输出}
$$

In [ ]:
class PINO1d(nn.Module):
    """
    Physics-Informed Neural Operator for 1D problems
    
    架构:
        输入 [u0, x] → Lifting → [FNO Block] × depth → Projection → 输出 u
    
    参数:
        in_channels: 输入通道数 (如: 初始条件 + 坐标)
        out_channels: 输出通道数 (如: PDE 解)
        modes: Fourier 模式数
        width: 隐藏层宽度
        depth: FNO 块数量
    """
    
    def __init__(self, in_channels=2, out_channels=1, modes=16, width=64, depth=4):
        super().__init__()
        self.modes = modes
        self.width = width
        self.depth = depth
        
        # ========== Lifting Layer ==========
        # 将输入 [batch, spatial, in_channels] 提升到 [batch, spatial, width]
        self.fc0 = nn.Linear(in_channels, width)
        
        # ========== FNO Blocks ==========
        self.fno_blocks = nn.ModuleList([
            FNOBlock1d(width, modes) for _ in range(depth)
        ])
        
        # ========== Projection Layers ==========
        # 将特征投影回输出空间
        self.fc1 = nn.Linear(width, 128)
        self.fc2 = nn.Linear(128, out_channels)
    
    def forward(self, x, grid=None):
        """
        参数:
            x: [batch, spatial, channels] 或 [batch, spatial]
            grid: [batch, spatial, 1] 可选的空间坐标
        
        返回:
            [batch, spatial, out_channels]
        """
        if x.dim() == 2:
            x = x.unsqueeze(-1)
        
        # 拼接空间坐标 (如果提供)
        if grid is not None:
            x = torch.cat([x, grid], dim=-1)
        
        # Lifting: 提升到高维
        x = self.fc0(x)  # [batch, spatial, width]
        x = x.permute(0, 2, 1)  # [batch, width, spatial]
        
        # FNO blocks: 特征变换
        for block in self.fno_blocks:
            x = block(x)
        
        # Projection: 投影回输出
        x = x.permute(0, 2, 1)  # [batch, spatial, width]
        x = F.gelu(self.fc1(x))
        x = self.fc2(x)
        
        return x


# 创建并测试模型
model = PINO1d(in_channels=2, out_channels=1, modes=16, width=64, depth=4)
model = model.to(device)

# 计算参数量
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("✅ PINO 模型创建成功!")
print(f"   模型参数量: {total_params:,}")
print(f"   可训练参数: {trainable_params:,}")

# 测试前向传播
batch_size, nx = 8, 128
test_u0 = torch.randn(batch_size, nx, 1).to(device)
test_grid = torch.linspace(0, 1, nx).view(1, nx, 1).expand(batch_size, -1, -1).to(device)
test_input = torch.cat([test_u0, test_grid], dim=-1)

with torch.no_grad():
    test_output = model(test_input)

print(f"   输入形状: {test_input.shape}")
print(f"   输出形状: {test_output.shape}")

## 6. 物理信息损失函数

PINO 的关键创新是在训练中加入**物理约束**。总损失函数为：

$$
\mathcal{L}_{PINO} = \mathcal{L}_{data} + \lambda_{pde} \mathcal{L}_{PDE} + \lambda_{bc} \mathcal{L}_{BC} + \lambda_{ic} \mathcal{L}_{IC}
$$

### 6.1 各损失项详解

1. **数据损失** $\mathcal{L}_{data}$：预测值与真实解的 MSE
   $$\mathcal{L}_{data} = \frac{1}{N}\sum_{i=1}^{N} \|u_{pred}^{(i)} - u_{true}^{(i)}\|^2$$

2. **PDE 残差损失** $\mathcal{L}_{PDE}$：PDE 方程在预测解上的残差
   $$\mathcal{L}_{PDE} = \frac{1}{N}\sum_{i=1}^{N} \|\mathcal{N}[u_{pred}^{(i)}]\|^2$$

3. **边界条件损失** $\mathcal{L}_{BC}$：边界条件的满足程度

4. **初始条件损失** $\mathcal{L}_{IC}$：初始条件的满足程度

### 6.2 以 Burgers 方程为例

Burgers 方程：
$$\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}$$

PDE 残差：
$$r = u_t + u \cdot u_x - \nu \cdot u_{xx}$$

In [ ]:
def compute_burgers_pde_loss(u, u0, x, t, nu=0.01):
    """
    计算 Burgers 方程的 PDE 残差损失
    
    PDE: ∂u/∂t + u·∂u/∂x = ν·∂²u/∂x²
    
    参数:
        u: [batch, nt, nx] 预测的时空解
        u0: [batch, nx] 初始条件
        x: [nx] 空间坐标
        t: [nt] 时间坐标
        nu: 粘性系数
    
    返回:
        pde_loss: PDE 残差的 MSE
        ic_loss: 初始条件损失
    """
    batch_size, nt, nx = u.shape
    dx = x[1] - x[0]
    dt = t[1] - t[0]
    
    # ========== 使用有限差分计算导数 ==========
    
    # 时间导数 ∂u/∂t (前向差分)
    u_t = torch.zeros_like(u)
    u_t[:, :-1, :] = (u[:, 1:, :] - u[:, :-1, :]) / dt
    u_t[:, -1, :] = u_t[:, -2, :]  # 外推边界
    
    # 空间导数 ∂u/∂x (中心差分，周期边界)
    u_x = torch.zeros_like(u)
    u_x[:, :, 1:-1] = (u[:, :, 2:] - u[:, :, :-2]) / (2 * dx)
    u_x[:, :, 0] = (u[:, :, 1] - u[:, :, -1]) / (2 * dx)
    u_x[:, :, -1] = (u[:, :, 0] - u[:, :, -2]) / (2 * dx)
    
    # 二阶空间导数 ∂²u/∂x² (中心差分)
    u_xx = torch.zeros_like(u)
    u_xx[:, :, 1:-1] = (u[:, :, 2:] - 2*u[:, :, 1:-1] + u[:, :, :-2]) / (dx**2)
    u_xx[:, :, 0] = (u[:, :, 1] - 2*u[:, :, 0] + u[:, :, -1]) / (dx**2)
    u_xx[:, :, -1] = (u[:, :, 0] - 2*u[:, :, -1] + u[:, :, -2]) / (dx**2)
    
    # ========== PDE 残差 ==========
    # r = u_t + u * u_x - nu * u_xx
    residual = u_t + u * u_x - nu * u_xx
    
    # PDE 损失 (排除边界)
    pde_loss = torch.mean(residual[:, 1:-1, 1:-1]**2)
    
    # ========== 初始条件损失 ==========
    ic_loss = F.mse_loss(u[:, 0, :], u0)
    
    return pde_loss, ic_loss


def pino_loss(pred, target, u0, x, t, nu=0.01, 
              lambda_data=1.0, lambda_pde=0.1, lambda_ic=1.0):
    """
    PINO 总损失函数
    
    L_PINO = λ_data * L_data + λ_pde * L_PDE + λ_ic * L_IC
    """
    # 数据损失
    data_loss = F.mse_loss(pred, target)
    
    # 物理损失
    pde_loss, ic_loss = compute_burgers_pde_loss(pred, u0, x, t, nu)
    
    # 总损失
    total_loss = lambda_data * data_loss + lambda_pde * pde_loss + lambda_ic * ic_loss
    
    return total_loss, {
        'data': data_loss.item(),
        'pde': pde_loss.item(),
        'ic': ic_loss.item()
    }


print("✅ 物理信息损失函数定义完成!")
print("   - compute_burgers_pde_loss: 计算 Burgers 方程 PDE 残差")
print("   - pino_loss: PINO 总损失函数 (数据 + 物理)")

## 7. 生成 Burgers 方程训练数据

我们使用 **1D Burgers 方程** 作为示例：

$$\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}$$

**问题设置**:
- 空间域: $x \in [-1, 1]$，周期边界条件
- 时间域: $t \in [0, 1]$
- 粘性系数: $\nu = 0.01$
- 初始条件: 随机正弦波组合

我们使用**谱方法 + RK4 时间积分**生成高精度参考解。

In [ ]:
def generate_burgers_data(n_samples=50, nx=128, nt=50, nu=0.01, L=2.0, T=1.0):
    """
    使用谱方法生成 Burgers 方程数据
    
    参数:
        n_samples: 样本数量
        nx: 空间网格点数
        nt: 时间网格点数
        nu: 粘性系数
        L: 空间域长度 [-L/2, L/2]
        T: 时间域长度 [0, T]
    
    返回:
        initial_conditions: [n_samples, nx] 初始条件
        solutions: [n_samples, nt, nx] 完整时空解
        x_grid: [nx] 空间网格
        t_grid: [nt] 时间网格
    """
    x = np.linspace(-L/2, L/2, nx, endpoint=False)
    t = np.linspace(0, T, nt)
    dx = x[1] - x[0]
    dt = t[1] - t[0]
    
    # 波数 (用于谱方法)
    k = np.fft.fftfreq(nx) * nx * 2 * np.pi / L
    
    initial_conditions = []
    solutions = []
    
    for _ in tqdm(range(n_samples), desc="🔄 生成数据"):
        # ========== 随机初始条件 ==========
        # 随机正弦波组合
        n_modes = np.random.randint(1, 4)
        u0 = np.zeros(nx)
        for m in range(n_modes):
            amp = np.random.uniform(-1, 1)
            freq = np.random.randint(1, 4)
            phase = np.random.uniform(0, 2*np.pi)
            u0 += amp * np.sin(freq * np.pi * x / (L/2) + phase)
        
        # 归一化到 [-1, 1]
        u0 = u0 / (np.max(np.abs(u0)) + 1e-8)
        
        # ========== RK4 + 谱方法求解 ==========
        u = np.zeros((nt, nx))
        u[0] = u0
        u_current = u0.copy()
        
        def rhs(u_):
            """Burgers 方程右端项: -u*u_x + nu*u_xx"""
            u_hat = np.fft.fft(u_)
            u_x = np.real(np.fft.ifft(1j * k * u_hat))
            u_xx = np.real(np.fft.ifft(-k**2 * u_hat))
            return -u_ * u_x + nu * u_xx
        
        for i in range(1, nt):
            # RK4 时间积分
            k1 = rhs(u_current)
            k2 = rhs(u_current + 0.5*dt*k1)
            k3 = rhs(u_current + 0.5*dt*k2)
            k4 = rhs(u_current + dt*k3)
            u_current = u_current + (dt/6) * (k1 + 2*k2 + 2*k3 + k4)
            u[i] = u_current
        
        initial_conditions.append(u0)
        solutions.append(u)
    
    return np.array(initial_conditions), np.array(solutions), x, t


# 生成数据
print("📊 开始生成 Burgers 方程数据...")
n_train, n_val, n_test = 40, 5, 5
nx, nt = 128, 50
nu = 0.01

all_ic, all_sol, x_grid, t_grid = generate_burgers_data(
    n_samples=n_train + n_val + n_test,
    nx=nx, nt=nt, nu=nu
)

# 划分数据集
train_ic, train_sol = all_ic[:n_train], all_sol[:n_train]
val_ic, val_sol = all_ic[n_train:n_train+n_val], all_sol[n_train:n_train+n_val]
test_ic, test_sol = all_ic[n_train+n_val:], all_sol[n_train+n_val:]

print(f"\n✅ 数据生成完成!")
print(f"   训练集: {n_train} 样本")
print(f"   验证集: {n_val} 样本")
print(f"   测试集: {n_test} 样本")
print(f"   空间网格: {nx} 点")
print(f"   时间网格: {nt} 点")

In [ ]:
# 可视化部分样本
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

for i in range(3):
    # 时空图
    im = axes[0, i].imshow(train_sol[i].T, aspect='auto', origin='lower',
                           extent=[0, t_grid[-1], x_grid[0], x_grid[-1]], 
                           cmap='RdBu_r')
    axes[0, i].set_xlabel('时间 t')
    axes[0, i].set_ylabel('空间 x')
    axes[0, i].set_title(f'样本 {i+1}: u(x,t)')
    plt.colorbar(im, ax=axes[0, i])
    
    # 不同时刻的切片
    for ti in [0, nt//4, nt//2, 3*nt//4, -1]:
        label = f't={t_grid[ti]:.2f}' if ti != -1 else f't={t_grid[-1]:.2f}'
        axes[1, i].plot(x_grid, train_sol[i, ti, :], label=label)
    axes[1, i].set_xlabel('空间 x')
    axes[1, i].set_ylabel('u')
    axes[1, i].set_title(f'样本 {i+1}: 时间演化')
    axes[1, i].legend(fontsize=8)

plt.tight_layout()
plt.suptitle('Burgers 方程数据样本', y=1.02, fontsize=14)
plt.show()

## 8. 训练 PINO 模型

现在我们来训练 PINO 模型，同时使用数据损失和物理损失。

### 训练策略

1. **损失权重调节**: 平衡数据损失和物理损失
2. **学习率调度**: 使用余弦退火调度
3. **早停机制**: 防止过拟合

In [ ]:
def train_pino_simple(model, train_ic, train_sol, val_ic, val_sol, 
                      x_grid, t_grid, epochs=200, lr=1e-3, nu=0.01):
    """
    简化版 PINO 训练函数
    
    这里我们训练模型从初始条件预测最终时刻的解
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    # 转换为 PyTorch 张量
    train_ic_t = torch.FloatTensor(train_ic).to(device)
    train_target = torch.FloatTensor(train_sol[:, -1, :]).to(device)  # 最终时刻
    val_ic_t = torch.FloatTensor(val_ic).to(device)
    val_target = torch.FloatTensor(val_sol[:, -1, :]).to(device)
    
    # 空间网格
    grid = torch.FloatTensor(x_grid).view(1, -1, 1).expand(train_ic.shape[0], -1, -1).to(device)
    val_grid = torch.FloatTensor(x_grid).view(1, -1, 1).expand(val_ic.shape[0], -1, -1).to(device)
    
    history = {'train_loss': [], 'val_loss': [], 'data_loss': [], 'physics_loss': []}
    best_val_loss = float('inf')
    
    for epoch in tqdm(range(epochs), desc="🎯 训练中"):
        model.train()
        optimizer.zero_grad()
        
        # 准备输入: [batch, nx, 2] = [u0, x]
        x_input = torch.cat([train_ic_t.unsqueeze(-1), grid], dim=-1)
        
        # 前向传播
        pred = model(x_input).squeeze(-1)  # [batch, nx]
        
        # 数据损失
        data_loss = F.mse_loss(pred, train_target)
        
        # 简化的物理损失: 边界条件 (周期边界)
        bc_loss = torch.mean((pred[:, 0] - pred[:, -1])**2)
        
        # 总损失
        loss = data_loss + 0.1 * bc_loss
        
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        # 验证
        model.eval()
        with torch.no_grad():
            val_input = torch.cat([val_ic_t.unsqueeze(-1), val_grid], dim=-1)
            val_pred = model(val_input).squeeze(-1)
            val_loss = F.mse_loss(val_pred, val_target)
        
        history['train_loss'].append(loss.item())
        history['val_loss'].append(val_loss.item())
        history['data_loss'].append(data_loss.item())
        history['physics_loss'].append(bc_loss.item())
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
        
        if (epoch + 1) % 50 == 0:
            print(f"  Epoch {epoch+1}: Train Loss = {loss.item():.6f}, "
                  f"Val Loss = {val_loss.item():.6f}")
    
    # 恢复最佳模型
    model.load_state_dict(best_model_state)
    
    return history


# 创建新模型并训练
print("🚀 开始训练 PINO 模型...")
model = PINO1d(in_channels=2, out_channels=1, modes=16, width=64, depth=4).to(device)

history = train_pino_simple(
    model, train_ic, train_sol, val_ic, val_sol,
    x_grid, t_grid, epochs=200, lr=1e-3, nu=nu
)

print("\n✅ 训练完成!")

In [ ]:
# 绘制训练曲线
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

epochs_range = range(1, len(history['train_loss']) + 1)

# 训练/验证损失
axes[0].semilogy(epochs_range, history['train_loss'], 'b-', label='训练损失', alpha=0.8)
axes[0].semilogy(epochs_range, history['val_loss'], 'r-', label='验证损失', alpha=0.8)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (log scale)')
axes[0].set_title('训练过程')
axes[0].legend()

# 损失分解
axes[1].semilogy(epochs_range, history['data_loss'], 'b-', label='数据损失', alpha=0.8)
axes[1].semilogy(epochs_range, history['physics_loss'], 'g-', label='物理损失', alpha=0.8)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss (log scale)')
axes[1].set_title('损失分解')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"最终训练损失: {history['train_loss'][-1]:.6f}")
print(f"最终验证损失: {history['val_loss'][-1]:.6f}")

## 9. 模型评估与可视化

让我们在测试集上评估模型性能，并可视化预测结果。

In [ ]:
def evaluate_model(model, test_ic, test_sol, x_grid):
    """评估模型在测试集上的性能"""
    model.eval()
    
    test_ic_t = torch.FloatTensor(test_ic).to(device)
    test_target = test_sol[:, -1, :]  # 最终时刻
    grid = torch.FloatTensor(x_grid).view(1, -1, 1).expand(test_ic.shape[0], -1, -1).to(device)
    
    with torch.no_grad():
        x_input = torch.cat([test_ic_t.unsqueeze(-1), grid], dim=-1)
        predictions = model(x_input).squeeze(-1).cpu().numpy()
    
    # 计算误差指标
    errors = []
    for i in range(len(test_ic)):
        error = np.linalg.norm(predictions[i] - test_target[i]) / np.linalg.norm(test_target[i])
        errors.append(error)
    
    return predictions, np.array(errors)


# 评估模型
predictions, errors = evaluate_model(model, test_ic, test_sol, x_grid)

print("📊 测试集评估结果:")
print(f"   平均相对 L2 误差: {errors.mean():.4e}")
print(f"   最大相对 L2 误差: {errors.max():.4e}")
print(f"   最小相对 L2 误差: {errors.min():.4e}")

In [ ]:
# 可视化预测结果
fig, axes = plt.subplots(2, n_test, figsize=(15, 8))

for i in range(n_test):
    # 预测 vs 真实
    axes[0, i].plot(x_grid, test_sol[i, -1, :], 'b-', linewidth=2, label='真实解')
    axes[0, i].plot(x_grid, predictions[i], 'r--', linewidth=2, label='PINO预测')
    axes[0, i].set_xlabel('x')
    axes[0, i].set_ylabel('u')
    axes[0, i].set_title(f'测试样本 {i+1} (t=1.0)')
    axes[0, i].legend(fontsize=8)
    
    # 误差
    error = np.abs(test_sol[i, -1, :] - predictions[i])
    axes[1, i].plot(x_grid, error, 'g-', linewidth=2)
    axes[1, i].set_xlabel('x')
    axes[1, i].set_ylabel('|误差|')
    axes[1, i].set_title(f'绝对误差 (Rel. L2: {errors[i]:.2e})')
    axes[1, i].set_ylim([0, error.max() * 1.2])

plt.tight_layout()
plt.suptitle('PINO 预测结果与真实解对比', y=1.02, fontsize=14)
plt.show()

## 10. 总结与展望

### 10.1 本教程回顾

在本教程中，我们学习了：

1. **PINO 的核心思想**: 结合神经算子和物理约束
2. **Fourier Neural Operator**: 在频域进行高效的全局学习
3. **物理信息损失**: 将 PDE 残差作为正则化项
4. **完整实现**: 从数据生成到模型训练和评估

### 10.2 PINO 的优势

| 优势 | 说明 |
|------|------|
| **数据效率** | 物理约束减少对标签数据的需求 |
| **泛化能力** | 可处理不同初始/边界条件 |
| **推理速度** | 训练后毫秒级预测 |
| **物理一致性** | PDE 残差保证物理合理性 |

### 10.3 进阶方向

- 🔬 **更复杂的 PDE**: Navier-Stokes、Maxwell 方程等
- 📈 **多分辨率学习**: 不同网格分辨率的泛化
- 🔧 **逆问题**: 从观测数据反演 PDE 参数
- ⚡ **加速技术**: 模型压缩、知识蒸馏

### 10.4 推荐阅读

1. Li et al. (2021). "Physics-Informed Neural Operator for Learning Partial Differential Equations"
2. Li et al. (2020). "Fourier Neural Operator for Parametric Partial Differential Equations"  
3. Raissi et al. (2019). "Physics-informed neural networks"

---

🎉 **恭喜完成本教程！** 下一步，你可以尝试：
- 运行 `burgers_pino.ipynb` 进行更深入的 Burgers 方程学习
- 探索 `darcy_pino.ipynb` 中的 2D 问题
- 查看 `examples/` 文件夹中的完整示例代码